In [ ]:
#uplaoding some test images to the environment
#https://drive.google.com/file/d/1A9e2tZ-EP7oLH1FU_KRfE7GXTS3-C40y/view?usp=sharing
#https://drive.google.com/file/d/1TbzCbx-dQTxG67duFJHxAwq-sEQyPARm/view?usp=sharing
#https://drive.google.com/file/d/1pCjy-GjGowyeE9KWe2_QHh2OMrkHnAMD/view?usp=sharing
!gdown --id 1pCjy-GjGowyeE9KWe2_QHh2OMrkHnAMD
!gdown --id 1TbzCbx-dQTxG67duFJHxAwq-sEQyPARm
!gdown --id 1A9e2tZ-EP7oLH1FU_KRfE7GXTS3-C40y

In [ ]:
import torchvision #computer vision hub of PyTorch
import torch
from PIL import Image
from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import numpy as np
from torchvision.io import read_image
from torchvision.transforms.functional import convert_image_dtype
import time

#utils
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
def save_result(imgs, filename):
  if not isinstance(imgs, list):
        imgs = [imgs]
  for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        img.save(filename+".jpg")

In [ ]:
#original torchvision classes names (this model is trained on COCO dataset)
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
#loading the Faster RCNN from torchvision (hub for computer vision models)
faster_rcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
#model layers (NB: the model is not sequential)
faster_rcnn_model.named_children

In [ ]:
#backbone componenet
model.backbone

In [ ]:
#region proposal component
model.rpn

In [ ]:
#load image (upload your image to the environment and )
image_int = read_image('test.jpg')

#model in evaluation mode
faster_rcnn_model.eval()

#prediction
batch = convert_image_dtype(image_int, dtype=torch.float)
batch = [batch]
predictions = faster_rcnn_model(batch)

In [ ]:
#model's raw output 
predictions

In [ ]:
#processing model output and drawing results
#tune the threshold as you wish
score_threshold = .4
drawings = draw_bounding_boxes(image_int, boxes=predictions[0]['boxes'][predictions[0]['scores'] > score_threshold ])
show(drawings)
save_result(drawings, "result")


In [ ]:
#vehicules only
score_threshold = .4
selected_boxes = predictions[0]['boxes'][(predictions[0]['scores'] > score_threshold) &  ((predictions[0]['labels']  < 11) & (predictions[0]['labels'] > 2) )]

drawings = draw_bounding_boxes(image_int, boxes=selected_boxes)
show(drawings)
save_result(drawings, "result_vehicules_only")

In [ ]:
#evaluating inference time
#make sure colab runtime is on GPU mode

input = torch.rand(1,3,600, 600)
if torch.cuda.is_available():
   print('evaluation is taking place on GPU:')
   input=input.to('cuda')
   model=faster_rcnn_model.to('cuda')
else:
   print('evaluation is taking place on CPU:')
faster_rcnn_model.eval()
faster_rcnn_model(input)
T = 0
for _ in range(5):
    t1 = time.time()
    faster_rcnn_model(input)
    t2 = time.time()
    T += (t2-t1)
T /= 5
print('Faster RCNN inference time : %f s ===> %f FPS' % (T, 1/T)) 
